In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

SETTING = kraft.read_json("setting.json")

PATH = make_path_dict(SETTING)

In [ ]:
if SETTING["signal_type"] != "context":

    raise

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
for element_type, dataframe in (
    ("feature", feature_x_sample_processed),
    ("sample", feature_x_sample_processed.T),
):

    if element_type not in SETTING["context_element_types"]:

        continue

    element_x_fit_parameter = pd.read_csv(
        PATH[f"{element_type}_x_fit_parameter.tsv"], sep="\t", index_col=0
    )

    context_matrix = kraft.make_context_matrix(
        dataframe,
        n_job=SETTING["n_job"],
        skew_t_pdf_fit_parameter=element_x_fit_parameter,
        output_file_path=PATH[f"feature_x_sample.{element_type}_context.tsv"],
    )

    context_matrix = pd.read_csv(
        PATH[f"feature_x_sample.{element_type}_context.tsv"], sep="\t", index_col=0
    )

    kraft.summarize_feature_x_sample(
        context_matrix,
        feature_x_sample_alias=f"{SETTING[f'{element_type}_alias']} Context",
        feature_x_sample_value_name="Context",
        plot_heat_map_max_size=SETTING["plot_heat_map_max_size"],
        plot_histogram_max_size=SETTING["plot_histogram_max_size"],
        plot_rug_max_size=SETTING["plot_rug_max_size"],
    )

    for context_type, clip_keyword_arguments in (
        ("negative", {"upper": 0}),
        ("positive", {"lower": 0}),
    ):

        element_context_sorted = (
            context_matrix.clip(**clip_keyword_arguments).sum(axis=1).sort_values()
        )

        elements_to_peek = (
            element_context_sorted.index & SETTING[f"{element_type}s_to_peek"]
        )

        kraft.plot_and_save(
            {
                "layout": {
                    "title": {"text": SETTING[f"{element_type}_alias"]},
                    "xaxis": {"title": "Rank"},
                    "yaxis": {"title": f"{context_type.title()} Context Sum"},
                    "annotations": [
                        {
                            "x": element,
                            "y": element_context_sorted[element],
                            "text": element,
                            "textangle": -90,
                            "font": {"size": 10},
                            "arrowhead": 2,
                            "arrowsize": 0.64,
                        }
                        for element in elements_to_peek
                    ],
                },
                "data": [
                    {
                        "type": "scatter",
                        "name": "All",
                        "x": element_context_sorted.index,
                        "y": element_context_sorted,
                        "text": element_context_sorted.index,
                    }
                ],
            },
            None,
        )

        if context_type == "negative":

            extreme_elements = element_context_sorted.index[:3]

        elif context_type == "positive":

            extreme_elements = element_context_sorted.index[-3:]

        for extreme_element in extreme_elements:

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                extreme_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            kraft.plot_context(
                dataframe.loc[extreme_element],
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                xaxis_title=SETTING["feature_x_sample_value_name"],
            )